In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats as stats

import seaborn as sns
#sns.set()
sns.set(color_codes=True)

ModuleNotFoundError: No module named 'scipy'

# Uncertainty and probability

In the first lab we used a normal distribution, let's look at plotting a normal distribution's PDF. Remember the normal (Gaussian) distribution is defined as

$$p(x; \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right).$$

In [1]:
def gaussian(x, mean, stdev):
    return np.exp(-((x-mean)**2)/(2*stdev**2))/(np.sqrt(2*np.pi)*stdev)

x = np.linspace(-15, 15, 250)
plt.plot(x, gaussian(x, 2, 4))

NameError: name 'np' is not defined